In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import backend.design_ratio as design_ratio
import backend.crack_detection as crack_detec
import backend.database_connection as db_conn
import backend.model_updating_daily as model_upt_dly

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import signal
import pickle
from pathlib import Path

In [4]:
import plotly.graph_objects as go

In [5]:
engine = db_conn.create_connection()

# Model updating

In [18]:
fre, dampopt, phi = get_model_analysis_data(engine)#model_upt_dly.get_model_analysis_data(engine)
#opt = model_upt_dly.opti(fre,phi)

#model_upt_dly.create_pickle_visualization()
print('Done Modal Update')

(5190, 9) (10381, 8) 5190 10382


ValueError: could not broadcast input array from shape (5192) into shape (5190)

## PSD 

In [6]:
def get_last_hour_accelerations(engine, df):
    date1 = datetime.utcfromtimestamp(df['date1'].unique()[0].tolist()/1e9)
    date2 = datetime.utcfromtimestamp(df['date2'].unique()[0].tolist()/1e9)
    get_data_acceleration_query = f'''SELECT * FROM "100hz_python_SIM" WHERE sampledatetime BETWEEN '{date1}' AND '{date2}';'''
    acceleration_data_interval = pd.read_sql(get_data_acceleration_query, engine)
    columns = ['AI1','AI2','AI3','AI4','AI5','AI10','AI11','AI12']
    accelerations_df_interval = acceleration_data_interval[columns]
    return accelerations_df_interval.to_numpy()

In [17]:
def get_model_analysis_data(engine):
    last_date_query = "SELECT * FROM modal_analysis_results ORDER BY date1 DESC LIMIT 1"
    last_date = pd.read_sql(last_date_query, engine).date1[0]
    last_hour_query = f"SELECT * FROM modal_analysis_results WHERE date1 = '{last_date}'"
    df_model_analysis_date = pd.read_sql(last_hour_query, engine)
    df_model_analysis_date.sort_values(by=['fre'], ascending=False,inplace = True)
    data_numpy = df_model_analysis_date.to_numpy()
    fre = data_numpy[:,1]
    dampopt = data_numpy[:,2]
    phi = np.array(df_model_analysis_date['phi'].values.tolist()).transpose()
    
    df_last = get_last_hour_accelerations(engine, df_model_analysis_date)
    df_len = int(len(df_last[:,1])/2)
    Nfft = len(df_last[:,1]/2)+1
    data_result = np.zeros((df_len,df_last.shape[1]+1))
    print(data_result.shape, df_last.shape, df_len, Nfft)
    for i in range(df_last.shape[1]):
        f,pxx = signal.welch(df_last[:,i],100,nfft = Nfft,nperseg=1024)
        data_result[:,i+1] = pxx
    data_result[:,0] = f
    psd = {
    'fre':fre,
    'dampopt':dampopt,
    'df':data_result
    }
    file = "test.pkl"
    f = open(Path.cwd()/file,"wb")
    pickle.dump(psd,f)
    f.close()
    return fre,dampopt,phi

In [117]:
test = pickle.load(open(file, "rb"))

In [160]:
traces = {}
for i in range(1,psd['df'].shape[1]):
    traces['traces_'+str(i)] = go.Scatter(x =psd['df'][:,0], y = 10*np.log(psd['df'][:,i]))
data = list(traces.values())
fig_psd = go.Figure(data)
fig_psd.update_xaxes(title_text = 'Frequency [Hz]', showgrid = False, zeroline = False)
fig_psd.update_yaxes(title_text = 'PSD [dB]', showgrid = False, zeroline = False)
fig_psd.update_layout(shapes=[
    dict(
      type= 'line',
      yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= psd['fre'][0], x1= psd['fre'][0]
    )
])
fig_psd.update_layout(
            {
            'showlegend' : False,
            'plot_bgcolor' : 'rgba(0,0,0,0)',
            'paper_bgcolor': 'rgba(0,0,0,0)'
})

# Design Ratio

In [5]:
design_ratio.create_files()

In [6]:
crack_detec.montecarlo_simulation(engine)

/home/ec2-user/shm_webpage/backend/crack_detection.py:73: ComplexWarning: Casting complex values to real discards the imaginary part
  da[i] = C[i]*cycles_sum_rainflow_j*(Y*dsig_eq_rand*math.sqrt(math.pi*a[i]))**m; #delta-a
